In [1]:
import xarray as xr
from esmtools.stats import corr
from esmtools.grid import convert_lon
import numpy as np
import pandas as pd
import os
import glob
from utils import *

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

# Calcular as anomalias de SSt na região 3.4 Nino

In [2]:
# Abrir o arquivo de SST
ersst = xr.open_dataset('sst.mean.nc')
# Slice dos dados de Temperatura da Superficie do Mar
ersst = ersst.sel(time=slice('1979-01-01', '2020-12-01'))
# Caculo das anomalias 
asst = ersst.groupby('time.month') - ersst.groupby('time.month').mean(dim='time') 
# Regiçao de Nino 3.4
nino34 = asst.sel(lon=slice(190, 240), lat=slice(5, -5)).mean(("lon", "lat"))
# Formato anual
nino34_y = nino34.sst.resample({'time':'YS'}).mean()

In [3]:
# Mudar o diretorio
os.chdir('/home/luiz/climate_change/ETCCDI_Extremes/Anual/')

In [4]:
# Definir caminho do shapefile
shp = '/home/luiz/Documentos/Shapefiles/South_America/South_America.shp'
# Criar uma lista com os indices extremos
ETCCDI_index = glob.glob('*nc')
ETCCDI_index.sort()

In [6]:
# Criar diretorio
path_cor_nc = '/home/luiz/Jupyter_Notebook/Python_Notebook/Climate_Extremes_DOC/Correlations_NC'
if os.path.exists(path_cor_nc):
    print(f'O diretório {path_cor_nc} já existe !')
else:
    print(f'Criando diretório {path_cor_nc}')
    os.mkdir(path_cor_nc)

O diretório /home/luiz/Jupyter_Notebook/Python_Notebook/Climate_Extremes_DOC/Correlations_NC já existe !


In [7]:
%%time
start_date = '1979-01-01'
end_date = '2020-01-01'
freq = '1YS'

for index in ETCCDI_index:
    ds = xr.open_dataset(index, decode_timedelta=False)
    ds = fix_encoding_time(ds, start=start_date, end=end_date, freq=freq)
    ds = convert_lon(ds, coord=get_lon_name(ds))
    # Recortar
    ds = mask_dataset(ds, shapefile_path=shp)
    
    # Remover variavel 'time_bnds'
    if 'time_bnds' in ds:
        ds = ds.drop_vars('time_bnds')
    else:
        pass
    
    # Pegar o nome do Indice Extremo
    index_name = index.split('_')[0]
    
    # Log
    print(f'{index_name.upper()} - Calculate Correlation between Nino 3.4 Region')
    
    # Calcular a correlação
    cor, p = corr(nino34_y, ds[getvar(ds)], dim='time', return_p=True)
    
    # Combinar parametros
    da = merge_params(cor, p, name_stats='cor', name_sig='sig')
    
    # Exportar arquivos
    print(f'Saving File {index_name}_COR_NINO34.nc')
    da.to_netcdf(f'{path_cor_nc}/' + f'{index_name}_COR_NINO34.nc')
    
    print('Done!')

CDD - Calculate Correlation between Nino 3.4 Region
Saving File cdd_COR_NINO34.nc
Done!
CSDI - Calculate Correlation between Nino 3.4 Region
Saving File csdi_COR_NINO34.nc
Done!
CWD - Calculate Correlation between Nino 3.4 Region
Saving File cwd_COR_NINO34.nc
Done!
DTR - Calculate Correlation between Nino 3.4 Region
Saving File dtr_COR_NINO34.nc
Done!
FD - Calculate Correlation between Nino 3.4 Region
Saving File fd_COR_NINO34.nc
Done!
PRCPTOT - Calculate Correlation between Nino 3.4 Region
Saving File prcptot_COR_NINO34.nc
Done!
R10MM - Calculate Correlation between Nino 3.4 Region
Saving File r10mm_COR_NINO34.nc
Done!
R20MM - Calculate Correlation between Nino 3.4 Region
Saving File r20mm_COR_NINO34.nc
Done!
R30MM - Calculate Correlation between Nino 3.4 Region
Saving File r30mm_COR_NINO34.nc
Done!
R95P - Calculate Correlation between Nino 3.4 Region
Saving File r95p_COR_NINO34.nc
Done!
R99P - Calculate Correlation between Nino 3.4 Region
Saving File r99p_COR_NINO34.nc
Done!
RX1DAY -